In [16]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer  # Stemmer específico para português
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from joblib import dump
from nltk.stem import RSLPStemmer
import string


In [17]:
df = pd.read_excel('df_reviews.xlsx')

In [18]:
df.shape

(110290, 8)

In [19]:
# Baixar recursos necessários do NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('rslp')  # Para o stemmer em português

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\henri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\henri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\henri\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [21]:
# Função para limpar e pré-processar o texto
def preprocess_text(text):
    if isinstance(text, str):
        # Converter para minúsculas
        text = text.lower()
        # Remover pontuação
        #    cria um mapeamento que apaga todos os caracteres de string.punctuation
        translator = str.maketrans('', '', string.punctuation)
        text = text.translate(translator)

        # Remover caracteres especiais e números
        text = re.sub(r'[^\w\s]', '', text)
        text = re.sub(r'\d+', '', text)
        # Tokenização
        tokens = nltk.word_tokenize(text)
        # Remover stopwords em português
        stop_words = set(stopwords.words('portuguese'))
        tokens = [word for word in tokens if word not in stop_words]
        # Stemming (redução das palavras ao seu radical)
        stemmer = RSLPStemmer()
        tokens = [stemmer.stem(word) for word in tokens]
        # Reconstruir texto
        return ' '.join(tokens)
    else:
        return ''

In [22]:
# Função para categorizar comentários usando regras baseadas em palavras-chave
def categorize_by_keywords(df, text_column):
    """
    Categoriza comentários com base em dicionários de palavras-chave.
    
    Args:
        df: DataFrame contendo os comentários
        text_column: Nome da coluna contendo o texto
    
    Returns:
        DataFrame com a coluna de categoria adicionada
    """
    # Definir categorias e palavras-chave relevantes para app bancário
    keywords_dict = {
        'Problemas de Login': ['login', 'senha', 'acesso', 'entrar', 'chave', 'seguranca', 'token', 'bloquead'],
        'Problemas Técnicos': ['erro', 'fecha', 'bug', 'trava', 'cai', 'instavel', 'lento', 'carrega', 'funcionou'],
        'Usabilidade': ['facil', 'dificil', 'complicad', 'simples', 'intuitivo', 'pratico', 'rapido', 'interface'],
        'Funcionalidades': ['funcao', 'extrato', 'transfer', 'pagamento', 'boleto', 'consulta', 'saldo', 'pix'],
        'Avaliação Geral': ['otimo', 'bom', 'excelente', 'ruim', 'pessimo',  'péssimo', 'horrivel', 'melhor', 'pior', 'recomend'],
        'Atualizações': ['atualiza', 'versao', 'nova', 'antigo', 'mudanca', 'melhora']
    }
    
    # Cria uma cópia do DataFrame para não modificar o original
    result_df = df.copy()
    
    # Adiciona uma coluna para a categoria
    result_df['categoria'] = 'Outros'
    
    # Itera sobre cada linha do DataFrame
    for idx, row in result_df.iterrows():
        text = str(row[text_column]).lower()
        
        # Verifica cada categoria
        for category, keywords in keywords_dict.items():
            # Se qualquer palavra-chave estiver presente no texto, atribui a categoria
            if any(keyword in text for keyword in keywords):
                result_df.at[idx, 'categoria'] = category
                break
                
    return result_df

In [23]:
# Função para analisar o sentimento do comentário com base na nota
def add_sentiment(df, score_column):
    """
    Adiciona uma coluna de sentimento com base na nota do comentário.
    
    Args:
        df: DataFrame contendo os comentários
        score_column: Nome da coluna contendo a nota (1-5)
    
    Returns:
        DataFrame com a coluna de sentimento adicionada
    """
    df['sentimento'] = df['score'].apply(lambda x: 
        'Positivo' if x >= 4 else 
        'Neutro' if x == 3 else 
        'Negativo')
    return df

In [24]:
# Função para treinar e avaliar modelos de machine learning
def train_ml_categorization(X_train, X_test, y_train, y_test):
    """
    Treina e avalia diferentes modelos de classificação.
    
    Args:
        X_train, X_test: Features de treino e teste
        y_train, y_test: Labels de treino e teste
    
    Returns:
        O melhor modelo treinado
    """
    # Define modelos para testar
    models = {
        'Naive Bayes': MultinomialNB(),
        'Regressão Logística': LogisticRegression(max_iter=1000),
        'Random Forest': RandomForestClassifier(),
        'SVM': LinearSVC()
    }
    
    best_score = 0
    best_model = None
    
    # Avalia cada modelo
    for name, model in models.items():
        print(f"\nTreinando {name}...")
        model.fit(X_train, y_train)
        score = model.score(X_test, y_test)
        print(f"Acurácia: {score:.4f}")
        
        y_pred = model.predict(X_test)
        print("\nRelatório de Classificação:")
        print(classification_report(y_test, y_pred))
        
        # Salva o melhor modelo
        if score > best_score:
            best_score = score
            best_model = model
    
    return best_model

In [25]:
# Função para visualizar a distribuição das categorias
def plot_categories_distribution(df, category_column):
    plt.figure(figsize=(12, 6))
    sns.countplot(y=df[category_column], order=df[category_column].value_counts().index)
    plt.title('Distribuição das Categorias')
    plt.xlabel('Número de Comentários')
    plt.ylabel('Categoria')
    plt.tight_layout()
    plt.savefig('categorias_distribuicao.png')
    plt.close()

In [26]:
# Função para visualizar cruzamento entre categoria e sentimento
def plot_category_sentiment(df, category_column, sentiment_column):
    plt.figure(figsize=(14, 8))
    ct = pd.crosstab(df[category_column], df[sentiment_column])
    ct_percent = ct.div(ct.sum(axis=1), axis=0)
    ct_percent.plot(kind='barh', stacked=True, colormap='viridis')
    plt.title('Distribuição de Sentimento por Categoria')
    plt.xlabel('Proporção')
    plt.ylabel('Categoria')
    plt.legend(title='Sentimento')
    plt.tight_layout()
    plt.savefig('categoria_sentimento.png')
    plt.close()

In [ ]:
# limpa valores ausentes
# df = df.dropna()

In [ ]:
# df.shape

In [28]:
# Função principal para processar o dataset
def main():
    # Carrega os dados
    # Ajuste o separador conforme necessário (parece ser um CSV ou TSV)
    try:
        df = pd.read_excel('df_reviews.xlsx')
        # df = df.dropna()
    except:
        # Tente outro separador se o primeiro não funcionar
        df = pd.read_excel('df_reviews.xlsx')
        # df = df.dropna()

    # Verifica as colunas disponíveis
    print("Colunas disponíveis no dataset:")
    print(df.columns.tolist())
    
    # Mostra as primeiras linhas do dataset
    print("\nPrimeiras linhas do dataset:")
    print(df.head())
    
    # Informações básicas
    print("\nInformações básicas sobre o dataset:")
    print(f"Número de comentários: {len(df)}")
    print(f"Distribuição das notas:")
    print(df['score'].value_counts().sort_index())
    
    # Pré-processamento dos comentários
    print("\nRealizando pré-processamento dos comentários...")
    df['comentario_processado'] = df['review'].apply(preprocess_text)
    
    # Adiciona sentimento com base na nota
    df = add_sentiment(df, 'score')
    
    # ABORDAGEM 1: Categorização baseada em palavras-chave
    print("\n==== ABORDAGEM 1: Categorização por palavras-chave ====")
    categorized_df = categorize_by_keywords(df, 'comentario_processado')
    
    print("Distribuição de categorias:")
    category_counts = categorized_df['categoria'].value_counts()
    print(category_counts)
    
    # Visualiza a distribuição das categorias
    plot_categories_distribution(categorized_df, 'categoria')
    
    # Visualiza relação entre categoria e sentimento
    plot_category_sentiment(categorized_df, 'categoria', 'sentimento')
    
    # ABORDAGEM 2: Categorização por machine learning
    print("\n==== ABORDAGEM 2: Categorização por machine learning ====")
    
    # Usar as categorias da abordagem por palavras-chave como rótulos para treinar o modelo
    X = categorized_df['comentario_processado']
    y = categorized_df['categoria']
    
    # Divide em conjuntos de treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Vetorização do texto usando TF-IDF
    print("\nVetorizando os textos...")
    vectorizer = TfidfVectorizer(max_features=5000)
    X_train_vec = vectorizer.fit_transform(X_train)
    X_test_vec = vectorizer.transform(X_test)
    
    # Treina e avalia os modelos
    best_model = train_ml_categorization(X_train_vec, X_test_vec, y_train, y_test)
    
    # Cria um pipeline com o melhor modelo
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(max_features=5000)),
        ('classifier', best_model)
    ])
    
    # Treina o pipeline com todos os dados
    pipeline.fit(X, y)
       
    # Salva o modelo treinado
    dump(pipeline, 'modelo_categorizacao_app_bradesco.joblib')
    print("Modelo salvo como 'modelo_categorizacao_app_bradesco.joblib'")
    
    # Salva o DataFrame categorizado
    categorized_df.to_csv('comentarios_categorizados.csv', index=False, encoding='utf-8-sig')
    categorized_df.to_excel('comentarios_categorizados.xlsx', index=False)
    
    print("Dataset categorizado salvo como 'comentarios_categorizados.csv'")


if __name__ == "__main__":
    main()

Colunas disponíveis no dataset:
['user', 'review', 'score', 'resposta', 'data', 'app_name', 'store', 'banco']

Primeiras linhas do dataset:
                   user                                             review  \
0  Um usuário do Google         Nao consigo abrir a conta, sempre dá erro.   
1  Um usuário do Google  Não consigo acessar minha conta no aplicativo,...   
2  Um usuário do Google                Um dos melhores app financeiros....   
3  Um usuário do Google  não é intuitivo, estou tentando excluir essa c...   
4  Um usuário do Google  estou a quatro dias tentando recuperar minha s...   

   score resposta                data                    app_name  \
0      2      NaN 2025-07-08 12:05:21  br.com.bradesco.netempresa   
1      1      NaN 2025-07-08 08:29:24  br.com.bradesco.netempresa   
2      5      NaN 2025-07-08 08:11:02  br.com.bradesco.netempresa   
3      1      NaN 2025-07-07 16:13:28  br.com.bradesco.netempresa   
4      2      NaN 2025-07-07 15:16:32  br.com.

c:\Users\henri\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\henri\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\henri\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                    precision    recall  f1-score   support

      Atualizações       0.00      0.00      0.00        45
   Avaliação Geral       0.98      0.89      0.93      8867
   Funcionalidades       0.85      0.48      0.61      1060
            Outros       0.89      1.00      0.94     20915
Problemas Técnicos       1.00      0.13      0.22       506
Problemas de Login       0.98      0.29      0.45       609
       Usabilidade       0.99      0.55      0.71      1085

          accuracy                           0.91     33087
         macro avg       0.81      0.48      0.55     33087
      weighted avg       0.91      0.91      0.90     33087


Treinando Regressão Logística...
Acurácia: 0.9803

Relatório de Classificação:
                    precision    recall  f1-score   support

      Atualizações       1.00      0.11      0.20        45
   Avaliação Geral       0.99      0.98      0.98      8867
   Funcionalidades       0.95      0.93      0.94      1060
            Outr

c:\Users\henri\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Acurácia: 0.9900

Relatório de Classificação:
                    precision    recall  f1-score   support

      Atualizações       0.95      0.44      0.61        45
   Avaliação Geral       1.00      0.99      0.99      8867
   Funcionalidades       0.98      0.96      0.97      1060
            Outros       0.99      1.00      0.99     20915
Problemas Técnicos       0.99      0.87      0.92       506
Problemas de Login       1.00      0.95      0.97       609
       Usabilidade       0.99      0.96      0.98      1085

          accuracy                           0.99     33087
         macro avg       0.99      0.88      0.92     33087
      weighted avg       0.99      0.99      0.99     33087



c:\Users\henri\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Modelo salvo como 'modelo_categorizacao_app_bradesco.joblib'
Dataset categorizado salvo como 'comentarios_categorizados.csv'


<Figure size 1400x800 with 0 Axes>

In [29]:
df.isna().sum().sort_values(ascending=False)

resposta    49703
review         24
user            0
score           0
data            0
app_name        0
store           0
banco           0
dtype: int64

In [ ]:
df.info()

In [ ]:

    # Exemplo de aplicação: categorizar novos comentários
    print("\n==== APLICAÇÃO: Categorizar novos comentários ====")
    
    # Exemplos de novos comentários (ajuste conforme necessário)
    novos_comentarios = [
        "Não consigo fazer login, pede a chave de segurança mas não reconhece",
        "App muito bom, fácil de usar e com todas as funções que preciso",
        "Fiz a atualização e agora fica travando quando tento fazer transferência",
        "Não consigo ver meu extrato, dá erro toda vez que tento acessar",
        "Excelente aplicativo, muito prático para gerenciar minha empresa"
    ]
    
    # Pré-processa os novos comentários
    novos_comentarios_proc = [preprocess_text(texto) for texto in novos_comentarios]
    
    # Prevê as categorias
    categorias_previstas = pipeline.predict(novos_comentarios_proc)
    
    # Exibe os resultados
    print("\nResultados da categorização:")
    for comentario, categoria in zip(novos_comentarios, categorias_previstas):
        print(f"Comentário: {comentario}")
        print(f"Categoria prevista: {categoria}\n")

In [ ]:

    # Dicas de insights que podem ser extraídos
    print("\n==== INSIGHTS POTENCIAIS ====")
    print("1. Categorias mais comuns de problemas reportados")
    print("2. Relação entre categorias e sentimento (notas)")
    print("3. Evolução temporal das categorias (análise de tendência)")
    print("4. Comparação entre versões do aplicativo")
    print("5. Palavras mais frequentes em cada categoria")